<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-research-and-practice/blob/main/text-similarity-works/03_fuzzywuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [5]:
import pandas as pd
import numpy as np
import re

import pdb

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

##Loading data

In [6]:
keyword_df = pd.read_csv("keywords.csv")
keyword_df.head()

,CATEGORY0,CATEGORY1,CATEGORY2,SYNID,KEYWORDS
0,Keywords,Respiratory,NaN,KW-rESPIRATORY01,Respiratory
1,Keywords,Alcohol,Alcohol,KW-ALCOHOL184,struggling with alcohol
2,Keywords,Alcohol,Alcohol,KW-ALCOHOL185,suspected alcohol abuse
3,Keywords,Alcohol,Alcohol,KW-ALCOHOL186,taking medication for alcohol
4,Keywords,Alcohol,Alcohol,KW-ALCOHOL187,tequilla


In [7]:
ocr_sample = pd.read_csv("ocr_page.csv", encoding="cp1252")
ocr_sample.head()

,PAGEID,OCR_CONTENT,LOAD_TIMESTAMP
0,104707,PACIFIC LIFE INSURANCE COMPANY\n\nLife Insuran...,26-JAN-22 07.04.57.000000000 PM
1,104708,\r\n\r\nTEMPORARY INSURANCE AGREEMENT (TIA)\r...,26-JAN-22 07.04.57.000000000 PM
2,104709,PACIFIC LIFE INSURANCE COMPANY\r\n\r\nLife Ins...,26-JAN-22 07.04.57.000000000 PM
3,104710,\n\nNON-FORFEITURE PREMIUM CESSATION OPTIONS ...,26-JAN-22 07.04.57.000000000 PM
4,104711,\n\nADDITIONAL POLICYOWNER (If more than one ...,26-JAN-22 07.04.57.000000000 PM


In [8]:
ocr_sample["OCR_CONTENT"][1]

' \r\n\r\nTEMPORARY INSURANCE AGREEMENT (TIA)\r\n\r\nQuali?cations\r\nTo qualify for temporary alcoholic hepatitis, the Proposed Insured(s) must be over 15 days of age or under age 70, and able to answer “No" to all of the\r\nquestions on the Temporary Insurance Agreement.\r\nLimits\r\n- The TIA is limited to $1,000,000 for individual products and $1,500,000 for Second-to-Die products.\r\n- For Monthly Bene?t Term Life Insurance only, the TIA is limited to the lesser ofthe Monthly Bene?t applied for or a Monthly Bene?t amount of $5,001\r\nInitial Premium\r\nThe minimum initial premium that can be submitted with the completed application and the TIA is either 1) 10% of the annual premium, or 2) greater of\r\none modal premium or three times the monthly initial minimum premium illustrated.\r\nSubmitting\r\n- It is preferred that the check for the TIA, application, and TIA form all have the same date.\r\n- If it is not possible to obtain a check at the time the application is completed, t

##Preparing query content

In [9]:
ocr_sample["OCR_CONTENT"][1].split("\n")[:10]

[' \r',
 '\r',
 'TEMPORARY INSURANCE AGREEMENT (TIA)\r',
 '\r',
 'Quali?cations\r',
 'To qualify for temporary alcoholic hepatitis, the Proposed Insured(s) must be over 15 days of age or under age 70, and able to answer “No" to all of the\r',
 'questions on the Temporary Insurance Agreement.\r',
 'Limits\r',
 '- The TIA is limited to $1,000,000 for individual products and $1,500,000 for Second-to-Die products.\r',
 '- For Monthly Bene?t Term Life Insurance only, the TIA is limited to the lesser ofthe Monthly Bene?t applied for or a Monthly Bene?t amount of $5,001\r']

In [10]:
# converting text to sentences
sentences_list = []
sentences = ocr_sample["OCR_CONTENT"][1].split("\n")
for sentence in sentences:
  if sentence != "":
    sentences_list.append(sentence)
sentences_list[:10]

[' \r',
 '\r',
 'TEMPORARY INSURANCE AGREEMENT (TIA)\r',
 '\r',
 'Quali?cations\r',
 'To qualify for temporary alcoholic hepatitis, the Proposed Insured(s) must be over 15 days of age or under age 70, and able to answer “No" to all of the\r',
 'questions on the Temporary Insurance Agreement.\r',
 'Limits\r',
 '- The TIA is limited to $1,000,000 for individual products and $1,500,000 for Second-to-Die products.\r',
 '- For Monthly Bene?t Term Life Insurance only, the TIA is limited to the lesser ofthe Monthly Bene?t applied for or a Monthly Bene?t amount of $5,001\r']

In [11]:
contents = ocr_sample["OCR_CONTENT"][1].replace("\r\n", "")
contents

' TEMPORARY INSURANCE AGREEMENT (TIA)Quali?cationsTo qualify for temporary alcoholic hepatitis, the Proposed Insured(s) must be over 15 days of age or under age 70, and able to answer “No" to all of thequestions on the Temporary Insurance Agreement.Limits- The TIA is limited to $1,000,000 for individual products and $1,500,000 for Second-to-Die products.- For Monthly Bene?t Term Life Insurance only, the TIA is limited to the lesser ofthe Monthly Bene?t applied for or a Monthly Bene?t amount of $5,001Initial PremiumThe minimum initial premium that can be submitted with the completed application and the TIA is either 1) 10% of the annual premium, or 2) greater ofone modal premium or three times the monthly initial minimum premium illustrated.Submitting- It is preferred that the check for the TIA, application, and TIA form all have the same date.- If it is not possible to obtain a check at the time the application is completed, the check may be dated up to 3 days later than the TIA and ap

##fuzzywuzzy

https://github.com/seatgeek/thefuzz

https://www.geeksforgeeks.org/fuzzywuzzy-python-library/

https://blog.ouseful.info/2017/09/04/simple-text-analysis-using-python-identifying-named-entities-tagging-fuzzy-string-matching-and-topic-modelling/

In [12]:
keyword_df["KEYWORDS"].head()

0                      Respiratory
1          struggling with alcohol
2          suspected alcohol abuse
3    taking medication for alcohol
4                         tequilla
Name: KEYWORDS, dtype: object

In [13]:
def make_keyword_list(df):
  keywords = []
  for _, row in df.iterrows():
    keywords.append(row["KEYWORDS"])
  return keywords

keywords = make_keyword_list(keyword_df)
keywords[:10]

['Respiratory',
 'struggling with alcohol',
 'suspected alcohol abuse',
 'taking medication for alcohol',
 'tequilla',
 'tipsy',
 'toxic effect of unspecified alcohol',
 'under the influence',
 'under the influence of alcohol',
 'unspecified alcohol disorder']

In [75]:
contents

' TEMPORARY INSURANCE AGREEMENT (TIA)Quali?cationsTo qualify for temporary alcoholic hepatitis, the Proposed Insured(s) must be over 15 days of age or under age 70, and able to answer “No" to all of thequestions on the Temporary Insurance Agreement.Limits- The TIA is limited to $1,000,000 for individual products and $1,500,000 for Second-to-Die products.- For Monthly Bene?t Term Life Insurance only, the TIA is limited to the lesser ofthe Monthly Bene?t applied for or a Monthly Bene?t amount of $5,001Initial PremiumThe minimum initial premium that can be submitted with the completed application and the TIA is either 1) 10% of the annual premium, or 2) greater ofone modal premium or three times the monthly initial minimum premium illustrated.Submitting- It is preferred that the check for the TIA, application, and TIA form all have the same date.- If it is not possible to obtain a check at the time the application is completed, the check may be dated up to 3 days later than the TIA and ap

In [14]:
process.extract(contents, keywords, limit=10)

[('alcoholic', 60),
 ('alcoholic hepatitis', 60),
 ('struggling with alcohol', 57),
 ('taking medication for alcohol', 57),
 ('toxic effect of unspecified alcohol', 57),
 ('under the influence', 57),
 ('under the influence of alcohol', 57),
 ('completed alcohol counselling', 57),
 ('evidence of alcohol involvement', 57),
 ('1 binge/year', 57)]

In [15]:
process.extractBests(contents, keywords)

[('alcoholic', 60),
 ('alcoholic hepatitis', 60),
 ('struggling with alcohol', 57),
 ('taking medication for alcohol', 57),
 ('toxic effect of unspecified alcohol', 57)]

In [16]:
process.extractBests(contents, keywords, score_cutoff=60)

[('alcoholic', 60), ('alcoholic hepatitis', 60)]

In [17]:
process.extractOne(contents, keywords)

('alcoholic', 60)

In [18]:
process.extractOne(contents, keywords, score_cutoff=50)

('alcoholic', 60)

##Sample search

In [19]:
sample_keywords = [
  "gallbladder", "gallbladder wall", "cholelithiasis", "pericholecystic fluid", "unremarkable", 
  "without evidence","normal in thickness", "unremarkable", "without evidence", "normal in thickness"
]

In [20]:
ocr_text = """
The gallbladder is unremarkable without evidence of cholelithiasis or pericholecystic fluid. The gallbladder wall is normal in thickness.
"""

process.extractBests(ocr_text, sample_keywords, limit=100)

[('pericholecystic fluid', 90),
 ('normal in thickness', 90),
 ('normal in thickness', 90),
 ('gallbladder', 60),
 ('cholelithiasis', 60),
 ('unremarkable', 60),
 ('without evidence', 60),
 ('unremarkable', 60),
 ('without evidence', 60),
 ('gallbladder wall', 57)]

In [21]:
ocr_text2 = """
The gallbladder is unremarkable without evidence of cholelithiasis or pericholecystic fluid. The gallbladder wall is normal in thickness.
"""

print(ocr_text2)
process.extract(ocr_text2, sample_keywords, scorer=fuzz.token_sort_ratio, limit=10)


The gallbladder is unremarkable without evidence of cholelithiasis or pericholecystic fluid. The gallbladder wall is normal in thickness.



[('pericholecystic fluid', 27),
 ('normal in thickness', 25),
 ('normal in thickness', 25),
 ('gallbladder wall', 21),
 ('without evidence', 21),
 ('without evidence', 21),
 ('cholelithiasis', 19),
 ('unremarkable', 16),
 ('unremarkable', 16),
 ('gallbladder', 15)]

In [22]:
process.extract(ocr_text2, sample_keywords, scorer=fuzz.token_set_ratio, limit=10)

[('gallbladder', 100),
 ('gallbladder wall', 100),
 ('cholelithiasis', 100),
 ('pericholecystic fluid', 100),
 ('unremarkable', 100),
 ('without evidence', 100),
 ('normal in thickness', 100),
 ('unremarkable', 100),
 ('without evidence', 100),
 ('normal in thickness', 100)]

In [23]:
ocr_text3 = """
The gallbladder is Unremarkable without evidence of cholelithiasis or pericholecystic fluid. The Gallbladder wall is normal in thickness.
"""

print(ocr_text3)
process.extract(ocr_text3, sample_keywords, scorer=fuzz.WRatio, limit=10)


The gallbladder is Unremarkable without evidence of cholelithiasis or pericholecystic fluid. The Gallbladder wall is normal in thickness.



[('pericholecystic fluid', 90),
 ('normal in thickness', 90),
 ('normal in thickness', 90),
 ('gallbladder', 60),
 ('cholelithiasis', 60),
 ('unremarkable', 60),
 ('without evidence', 60),
 ('unremarkable', 60),
 ('without evidence', 60),
 ('gallbladder wall', 57)]

In [24]:
ocr_text4 = """
The gallbladder is unremarkable without evidence of cholelithiasis or pericholecystic fluid. The gallbladder wall is normal in thickness.
"""

print(ocr_text4)
process.extractBests(ocr_text4, sample_keywords, scorer=fuzz.ratio, limit=10)


The gallbladder is unremarkable without evidence of cholelithiasis or pericholecystic fluid. The gallbladder wall is normal in thickness.



[('pericholecystic fluid', 27),
 ('normal in thickness', 25),
 ('normal in thickness', 25),
 ('gallbladder wall', 21),
 ('without evidence', 21),
 ('without evidence', 21),
 ('cholelithiasis', 19),
 ('unremarkable', 16),
 ('unremarkable', 16),
 ('gallbladder', 15)]

In [25]:
# Exact match
fuzz.ratio('The gallbladder is unremarkable without evidence of cholelithiasis or pericholecystic fluid. The gallbladder wall is normal in thickness.', 'gallbladder')

15

In [26]:
1 / 1

1.0

In [27]:
process.extractBests(ocr_text3, sample_keywords, limit=10)

[('pericholecystic fluid', 90),
 ('normal in thickness', 90),
 ('normal in thickness', 90),
 ('gallbladder', 60),
 ('cholelithiasis', 60),
 ('unremarkable', 60),
 ('without evidence', 60),
 ('unremarkable', 60),
 ('without evidence', 60),
 ('gallbladder wall', 57)]

In [28]:
terms=['Houses of Parliament', 'Diane Abbott', 'Boris Johnson']
 
q= "Diane Jafer, Theresa May and Boris Johnstone were in the Houses of Parliament the other day"
process.extract(q,terms, scorer=fuzz.WRatio)

[('Houses of Parliament', 90), ('Diane Abbott', 86), ('Boris Johnson', 86)]

In [29]:
process.extract(ocr_text, sample_keywords)

[('pericholecystic fluid', 90),
 ('normal in thickness', 90),
 ('normal in thickness', 90),
 ('gallbladder', 60),
 ('cholelithiasis', 60)]

In [30]:
set(process.extract(ocr_text, sample_keywords, limit=10))

{('cholelithiasis', 60),
 ('gallbladder', 60),
 ('gallbladder wall', 57),
 ('normal in thickness', 90),
 ('pericholecystic fluid', 90),
 ('unremarkable', 60),
 ('without evidence', 60)}

In [31]:
sample_keyword_groups ={
    "site_location": ["gallbladder", "gallbladder wall"],
    "disease_injury_condition": ["cholelithiasis", "pericholecystic fluid"],
    "findings_observations": ["unremarkable", "without evidence","normal in thickness"],
    "codes": ["ICD10", "LOINC", "RXNORM", "CPT"]
}

In [32]:
for item in sample_keyword_groups.items():
  print(set(process.extractBests(ocr_text, item[1])))

{('gallbladder', 60), ('gallbladder wall', 57)}
{('cholelithiasis', 60), ('pericholecystic fluid', 90)}
{('without evidence', 60), ('unremarkable', 60), ('normal in thickness', 90)}
{('CPT', 20), ('RXNORM', 40), ('LOINC', 36), ('ICD10', 24)}


##Regex Serach

In [62]:
sample_keywords = [
  "gallbladder", "gallbladder wall", "cholelithiasis", "pericholecystic fluid", "unremarkable", 
  "without evidence","normal in thickness", "unremarkable", "without evidence", "normal in thickness"
]

In [73]:
ocr_text = """
The gallbladder is unremarkable without evidence of gallbladder cholelithiasis or pericholecystic fluid. The gallbladder wall is normal in thickness.
"""

In [ ]:
for keyword in sample_keywords:
    print('Looking for "%s"' % (keyword), end=' ')
    #if re.search(keyword, ocr_text):
    #print('found a match!')
    print(re.search(keyword, ocr_text))
    print(re.search(keyword, ocr_text).span.value)

In [74]:
for keyword in sample_keywords:
  print('Looking for "%s"' % (keyword), end=' ')
  match = re.search(keyword, ocr_text)
  if match != None: 
    print("Match at index % s, % s" % (match.start(), match.end()))
    print("Full match: % s" % (match.group(0)))

Looking for "gallbladder" Match at index 5, 16
Full match: gallbladder
Looking for "gallbladder wall" Match at index 110, 126
Full match: gallbladder wall
Looking for "cholelithiasis" Match at index 65, 79
Full match: cholelithiasis
Looking for "pericholecystic fluid" Match at index 83, 104
Full match: pericholecystic fluid
Looking for "unremarkable" Match at index 20, 32
Full match: unremarkable
Looking for "without evidence" Match at index 33, 49
Full match: without evidence
Looking for "normal in thickness" Match at index 130, 149
Full match: normal in thickness
Looking for "unremarkable" Match at index 20, 32
Full match: unremarkable
Looking for "without evidence" Match at index 33, 49
Full match: without evidence
Looking for "normal in thickness" Match at index 130, 149
Full match: normal in thickness


In [60]:
ocr_text[5:16]

'gallbladder'

In [61]:
ocr_text[98: 114]

'gallbladder wall'

##Simple ratio usage

In [33]:
# Exact match
fuzz.ratio('geeksforgeeks', 'geeksgeeks')

87

In [34]:
fuzz.ratio('GeeksforGeeks', 'GeeksforGeeks') 

100

In [35]:
fuzz.ratio('geeks for geeks', 'Geeks For Geeks ')

77

In [36]:
fuzz.ratio(sample_keywords, ocr_text)

53

In [37]:
fuzz.partial_ratio("geeks for geeks", "geeks for geeks!")

100

In [38]:
fuzz.partial_ratio("geeks for geeks", "geeks geeks")

64

##Token Sort Ratio

In [39]:
# This gives 100 as every word is same, irrespective of the position 
fuzz.token_sort_ratio("geeks for geeks", "for geeks geeks")

100

In [40]:
fuzz.token_sort_ratio("geeks for geeks", "geeks for for geeks")

88

In [41]:
# Score comes 100 in second case because token_set_ratio considers duplicate words as a single word.
fuzz.token_set_ratio("geeks for geeks", "geeks for for geeks")

100

##Process module

In [42]:
query = 'geeks for geeks'
choices = ['geek for geek', 'geek geek', 'g. for geeks']

In [43]:
# Get a list of matches ordered by score, default limit to 5
process.extract(query, choices)

[('g. for geeks', 95), ('geek for geek', 93), ('geek geek', 86)]

In [44]:
# If we want only the top one
process.extractOne(query, choices)

('g. for geeks', 95)

##WRatio

There is also one more ratio which is used often called WRatio, sometimes its better to use WRatio instead of simple ratio as WRatio handles lower and upper cases and some other parameters too.

In [45]:
fuzz.WRatio('geeks for geeks', 'Geeks For Geeks')

100

In [46]:
fuzz.WRatio('geeks for geeks!!!','geeks for geeks')

100

In [47]:
# whereas simple ratio will give for above case
fuzz.ratio('geeks for geeks!!!','geeks for geeks')

91